In [1]:
import os 

import pandas as pd
import numpy as np

In [6]:
print(os.getcwd())
%cd ../../..

/Users/elizagoler/Documents/RecessionFertility/source/derived/county_gfrs
/Users/elizagoler/Documents/RecessionFertility


## From 2003 onwards, CDC Wonder has annual county-level fertility rates. I use those directly. ##

Read in data from 2007-2012

In [8]:
frates_2007_12 = pd.read_csv('datastore/raw/cdc_wonder/data/cdc_wonder_2007-2012.csv')

Clean data types and remove decimal points on county codes and years:

In [ ]:
#frates_2007_12.dtypes
frates_2007_12['County Code'] = frates_2007_12['County Code'].astype(str).str.replace(r'\.0$', '', regex=True)
frates_2007_12['Year'] = frates_2007_12['Year'].astype(str).str.replace(r'\.0$', '', regex=True)

In [16]:
frates_2007_12.head()

,Notes,County,County Code,Year,Year Code,Births,Total Population,Birth Rate,Female Population,Fertility Rate
0,NaN,"Baldwin County, AL",1003,2007,2007.0,2281.0,172404.0,13.23,31772.0,71.79
1,NaN,"Baldwin County, AL",1003,2008,2008.0,2289.0,175827.0,13.02,32118.0,71.27
2,NaN,"Baldwin County, AL",1003,2009,2009.0,2125.0,179406.0,11.84,32356.0,65.68
3,NaN,"Baldwin County, AL",1003,2010,2010.0,2167.0,182265.0,11.89,32715.0,66.24
4,NaN,"Baldwin County, AL",1003,2011,2011.0,2157.0,186717.0,11.55,33387.0,64.61


Read in data file for 2003-2006:

In [17]:
frates_2003_06 = pd.read_csv('datastore/raw/cdc_wonder/data/cdc_wonder_2003-2006.csv')

Do the same cleaning here as I did above:

In [19]:
frates_2003_06['County Code'] = frates_2003_06['County Code'].astype(str).str.replace(r'\.0$', '', regex=True)
frates_2003_06['Year'] = frates_2003_06['Year'].astype(str).str.replace(r'\.0$', '', regex=True)

Concatenate the 2003-2012 data:

In [22]:
frates_2003_12 = pd.concat([frates_2003_06, frates_2007_12])

In [23]:
frates_2003_12.head()

,Notes,County,County Code,Year,Year Code,Births,Total Population,Birth Rate,Female Population,Fertility Rate
0,NaN,"Baldwin County, AL",1003,2003,2003.0,1860.0,151509.0,12.28,28759.0,64.68
1,NaN,"Baldwin County, AL",1003,2004,2004.0,1872.0,156266.0,11.98,29331.0,63.82
2,NaN,"Baldwin County, AL",1003,2005,2005.0,2099.0,162183.0,12.94,30201.0,69.50
3,NaN,"Baldwin County, AL",1003,2006,2006.0,2245.0,168121.0,13.35,31071.0,72.25
4,Total,"Baldwin County, AL",1003,nan,NaN,8076.0,638079.0,12.66,119362.0,67.66


## For 1995 to 2002, I use the annual count of births by county from CDC Wonder (numerator). I use SEER data to get the population of women ages 15-44 (denominator). ##

In [25]:
birth_counts_1995_2002 = pd.read_csv('datastore/raw/cdc_wonder/data/cdc_wonder_1995-2002.csv')

In [26]:
birth_counts_1995_2002.head()

,Notes,County,County Code,Year,Year Code,Births
0,NaN,"Calhoun County, AL",1015.0,1995.0,1995.0,1520.0
1,NaN,"Calhoun County, AL",1015.0,1996.0,1996.0,1632.0
2,NaN,"Calhoun County, AL",1015.0,1997.0,1997.0,1588.0
3,NaN,"Calhoun County, AL",1015.0,1998.0,1998.0,1657.0
4,NaN,"Calhoun County, AL",1015.0,1999.0,1999.0,1586.0


In [32]:
# 1. Drop 'Year Code' column
if 'Year Code' in birth_counts_1995_2002.columns:
    birth_counts_1995_2002 = birth_counts_1995_2002.drop(columns=['Year Code'])

# 2. Remove '.0' from 'County Code' and 'Year' columns
for col in ['County Code', 'Year']:
    birth_counts_1995_2002[col] = (
        birth_counts_1995_2002[col]
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
    )

# 3. Clean 'Births': set missing, convert to int (no decimals)
# First, turn to str and identify missing
births_col = (
    birth_counts_1995_2002['Births']
    .astype(str)
    .replace(['nan', 'NaN', '', 'None'], pd.NA)
)
# Remove ".0", then convert
births_col = births_col.mask(births_col.isna(), None).str.replace(r'\.0$', '', regex=True)
# Convert to Int64 to allow NA, ensuring no decimals displayed
birth_counts_1995_2002['Births'] = pd.to_numeric(births_col, errors='coerce').astype('Int64')

# 4. Ensure correct dtypes for County Code and Year
birth_counts_1995_2002['County Code'] = birth_counts_1995_2002['County Code'].astype(str)
birth_counts_1995_2002['Year'] = birth_counts_1995_2002['Year'].astype(str)


In [33]:
birth_counts_1995_2002.head()

,Notes,County,County Code,Year,Births
0,NaN,"Calhoun County, AL",1015,1995,1520
1,NaN,"Calhoun County, AL",1015,1996,1632
2,NaN,"Calhoun County, AL",1015,1997,1588
3,NaN,"Calhoun County, AL",1015,1998,1657
4,NaN,"Calhoun County, AL",1015,1999,1586


In [34]:
birth_counts_1995_2002.dtypes

Notes          object
County         object
County Code    object
Year           object
Births          Int64
dtype: object

In [38]:
# Print one representative row for each state (state = last two letters of "County")
four_digit_rows = birth_counts_1995_2002[birth_counts_1995_2002['County Code'].str.len() == 4].copy()
four_digit_rows['State'] = four_digit_rows['County'].str[-2:]
print(four_digit_rows.groupby('State', as_index=False).head(1))


    Notes                 County County Code  Year  Births State
0     NaN     Calhoun County, AL        1015  1995    1520    AL
65    NaN  Anchorage Borough, AK        2020  1995    4453    AK
81    NaN    Maricopa County, AZ        4013  1995   44053    AZ
129   NaN     Pulaski County, AR        5119  1995    5636    AR
153   NaN     Alameda County, CA        6001  1995   20960    CA
433   NaN       Adams County, CO        8001  1995    4861    CO
513   NaN   Fairfield County, CT        9001  1995   12203    CT


Seems like the four digit county codes are missing a zero at the beginning

In [39]:
# Pad County Code with leading zero if it is four digits
birth_counts_1995_2002.loc[
    birth_counts_1995_2002['County Code'].str.len() == 4, 'County Code'
] = (
    '0' + birth_counts_1995_2002.loc[
        birth_counts_1995_2002['County Code'].str.len() == 4, 'County Code'
    ]
)


In [45]:
# Make all column names lowercase
birth_counts_1995_2002.columns = [col.lower() for col in birth_counts_1995_2002.columns]

# Rename 'county code' to 'county_fips'
birth_counts_1995_2002 = birth_counts_1995_2002.rename(columns={'county code': 'county_fips'})


In [46]:
birth_counts_1995_2002.head()

,notes,county,county_fips,year,births
0,NaN,"Calhoun County, AL",01015,1995,1520
1,NaN,"Calhoun County, AL",01015,1996,1632
2,NaN,"Calhoun County, AL",01015,1997,1588
3,NaN,"Calhoun County, AL",01015,1998,1657
4,NaN,"Calhoun County, AL",01015,1999,1586


Bringing in the population data for the relevant years:

In [50]:
gfr_women_pop_1969_2023_df = pd.read_csv('datastore/output/seer_population/gfr_women_pop_1969-2023.csv')

In [52]:
gfr_women_pop_1969_2023_df = gfr_women_pop_1969_2023_df.drop(columns=['fips_county_suffix'])


In [53]:
gfr_women_pop_1969_2003_df.head()

,year,county_fips,gfr_women_pop
0,1969,1001,4941
1,1969,1003,11553
2,1969,1005,4311
3,1969,1007,2557
4,1969,1009,5228


In [54]:
gfr_women_pop_1995_2002_df = gfr_women_pop_1969_2003_df[gfr_women_pop_1969_2023_df['year'].isin(range(1995, 2003))]

In [127]:
gfr_women_pop_1995_2002_df['year'] = gfr_women_pop_1995_2002_df['year'].astype(str)
gfr_women_pop_1995_2002_df['county_fips'] = (
    gfr_women_pop_1995_2002_df['county_fips'].astype(str).str.zfill(5)
)
gfr_women_pop_1995_2002_df.dtypes

/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_81350/2395851824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gfr_women_pop_1995_2002_df['year'] = gfr_women_pop_1995_2002_df['year'].astype(str)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_81350/2395851824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gfr_women_pop_1995_2002_df['county_fips'] = (


year             object
county_fips      object
gfr_women_pop     int64
dtype: object

In [128]:
gfr_women_pop_1995_2002_df.head()

,year,county_fips,gfr_women_pop
0,1969,01001,4941
1,1969,01003,11553
2,1969,01005,4311
3,1969,01007,2557
4,1969,01009,5228


In [129]:
gfr_1995_2002_df = pd.merge(
    birth_counts_1995_2002,
    gfr_women_pop_1995_2002_df,
    how='left',
    left_on=['year', 'county_fips'],
    right_on=['year', 'county_fips']
)


In [130]:
num_counties_nan_gfr = gfr_1995_2002_df[
    (gfr_1995_2002_df['year'].astype(float) > 2005) & (gfr_1995_2002_df['gfr_women_pop'].isna())
]['county_fips'].nunique()
num_counties_nan_gfr


0

In [131]:
gfr_1995_2002_df.head()

,notes,county,county_fips,year,births,gfr_women_pop
0,NaN,"Calhoun County, AL",01015,1995,1520,NaN
1,NaN,"Calhoun County, AL",01015,1996,1632,NaN
2,NaN,"Calhoun County, AL",01015,1997,1588,NaN
3,NaN,"Calhoun County, AL",01015,1998,1657,NaN
4,NaN,"Calhoun County, AL",01015,1999,1586,NaN


## There's a lot of missing stuff and I will deal with it later

In [132]:
# List of excluded county names (accounting for state names in the 'county' field)
excluded_county_names = [
    "Yuma County, AZ",
    "Adams County, CO",
    "Boulder County, CO",
    "Jefferson County, CO",
    "Weld County, CO",
    "Dade County, FL",
    "Los Alamos County, NM",
    "Valencia County, NM",
    "New York County, NY",
    "Washabaugh County, SD",
    "Alleghany County, VA",
    "Arlington County, VA",
    "Henry County, VA",
    "James City County, VA",
    "Rockingham County, VA",
    "Spotsylvania County, VA",
    "Stafford County, VA",
    "York County, VA",
    "Alexandria City, VA",
    "Clifton Forge City, VA",
    "Covington City, VA",
    "Falls Church City, VA",
    "Fredericksburg City, VA",
    "Galax City, VA",
    "Harrisonburg City, VA",
    "Martinsville City, VA",
    "Norton City, VA",
    "Williamsburg City, VA"
]

# Find counties not in the above list with NaN gfr_women_pop, and display all pairs as a list of tuples
nan_gfr_mask = gfr_1995_2002_df['gfr_women_pop'].isna()
not_excluded_mask = ~gfr_1995_2002_df['county'].isin(excluded_county_names)
unmatched_nan_counties_df = gfr_1995_2002_df.loc[nan_gfr_mask & not_excluded_mask, ['county', 'year']].drop_duplicates()

# Show all such (county, year) as a list
county_year_list = list(unmatched_nan_counties_df.itertuples(index=False, name=None))
print("County and year pairs with NaN gfr_women_pop not in the excluded list (as a list):")
print(county_year_list)


County and year pairs with NaN gfr_women_pop not in the excluded list (as a list):
[('Calhoun County, AL', '1995'), ('Calhoun County, AL', '1996'), ('Calhoun County, AL', '1997'), ('Calhoun County, AL', '1998'), ('Calhoun County, AL', '1999'), ('Calhoun County, AL', '2000'), ('Calhoun County, AL', '2001'), ('Calhoun County, AL', '2002'), ('Jefferson County, AL', '1995'), ('Jefferson County, AL', '1996'), ('Jefferson County, AL', '1997'), ('Jefferson County, AL', '1998'), ('Jefferson County, AL', '1999'), ('Jefferson County, AL', '2000'), ('Jefferson County, AL', '2001'), ('Jefferson County, AL', '2002'), ('Madison County, AL', '1995'), ('Madison County, AL', '1996'), ('Madison County, AL', '1997'), ('Madison County, AL', '1998'), ('Madison County, AL', '1999'), ('Madison County, AL', '2000'), ('Madison County, AL', '2001'), ('Madison County, AL', '2002'), ('Mobile County, AL', '1995'), ('Mobile County, AL', '1996'), ('Mobile County, AL', '1997'), ('Mobile County, AL', '1998'), ('Mobile

In [ ]:
# Calculate fertility_rate
gfr_1995_2002_df['fertility_rate'] = (gfr_1995_2002_df['births'] / gfr_1995_2002_df['gfr_women_pop']) * 1000

In [ ]:
# Count how many rows have fertility_rate as NA (null)
na_count = gfr_1995_2002_df['fertility_rate'].isna().sum()
print(f"Number of counties with fertility_rate = NA: {na_count}")

Number of counties with fertility_rate = NA: 984


In [86]:
# Show some rows where fertility_rate is not NA
print(gfr_1995_2002_df[gfr_1995_2002_df['fertility_rate'].notna()].head(10))

    notes                 county county_fips  year  births  gfr_women_pop  \
577   NaN        Kent County, DE       10001  1995    1738        28496.0   
578   NaN        Kent County, DE       10001  1996    1866        28574.0   
579   NaN        Kent County, DE       10001  1997    1715        28634.0   
580   NaN        Kent County, DE       10001  1998    1881        28777.0   
581   NaN        Kent County, DE       10001  1999    1905        28861.0   
582   NaN        Kent County, DE       10001  2000    1942        28970.0   
583   NaN        Kent County, DE       10001  2001    1883        29138.0   
584   NaN        Kent County, DE       10001  2002    1886        29675.0   
585   NaN  New Castle County, DE       10003  1995    6855       113933.0   
586   NaN  New Castle County, DE       10003  1996    6584       114753.0   

     fertility_rate  
577       60.991016  
578       65.304123  
579       59.893833  
580       65.364701  
581       66.006029  
582       67.034864 

**Now computing for years 1968-1995**

**First compute births by county x year**

In [102]:
nvss_68_78 = pd.read_parquet("datastore/output/nvss/nvss_68_78.parquet")
nvss_79_88 = pd.read_parquet("datastore/output/nvss/nvss_79_88.parquet")
nvss_89_93 = pd.read_parquet("datastore/output/nvss/nvss_89_93.parquet")
nvss_94_95 = pd.read_parquet("datastore/output/nvss/nvss_94_95.parquet")

#pd.read_parquet("datastore/output/nvss/nvss_79_88.parquet")

In [104]:
nvss_94_95.columns

Index(['datayear', 'cntyres', 'cityres'], dtype='object')

In [105]:
nvss_68_95 = pd.concat([nvss_68_78, nvss_79_88, nvss_89_93, nvss_94_95], ignore_index=True)
nvss_68_95.columns


Index(['datayear', 'cntyres', 'cityres'], dtype='object')

In [106]:
nvss_68_95.to_parquet("datastore/output/nvss/nvss_68_95.parquet", index=False)

In [107]:
nvss_68_95.head()

,datayear,cntyres,cityres
0,1968,1020,999
1,1968,1001,999
2,1968,1001,999
3,1968,1026,999
4,1968,1001,999


Each row corresponds to one birth record. So now, to get the total number of births per county x year, I need to group by year and county and sum the number of recrods.

In [108]:
# Group the nvss_68_95 dataframe by 'datayear' and 'cntyres' to count number of birth records per county x year
birth_counts_df = nvss_68_95.groupby(['datayear', 'cntyres']).size().reset_index(name='num_births')


In [ ]:
print(len(birth_counts_df['cntyres'].unique()))
# Convert cntyres to string to avoid type conversion issues
birth_counts_df['cntyres'] = birth_counts_df['cntyres'].astype(str)


3609


In [122]:
births_68_95 = birth_counts_df

**Putting 1968 to 1995 birth data into a parquet so don't have to do the above steps every time**

In [135]:
births_68_95.rename(columns={'datayear': 'year', 'cntyres': 'county_fips'}, inplace=True)
births_68_95

,year,county_fips,num_births
0,1968,10001,973
1,1968,10002,89
2,1968,10003,688
3,1968,10004,129
4,1968,10005,2120
...,...,...,...
75105,1995,7008,467
75106,1995,8001,1644
75107,1995,8002,6244
75108,1995,8003,1576


In [ ]:
# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

output_dir = "datastore/output/nvss"
births_68_95.to_parquet(os.path.join(output_dir, "nvss_68_95.parquet"), index=False)

In [125]:
gfr_women_pop_1995_2002_df = gfr_women_pop_1969_2003_df[gfr_women_pop_1969_2023_df['year'].isin(range(1968, 1995))]

In [133]:
gfr_women_pop_1995_2002_df.head()

,year,county_fips,gfr_women_pop
0,1969,01001,4941
1,1969,01003,11553
2,1969,01005,4311
3,1969,01007,2557
4,1969,01009,5228
